In [8]:
#!pip install -e ../
#get path of all stata files
import os
import re
import pandas as pd
RAW_DATA_DIR=  os.path.join('..', 'data','raw')  
GLES_DIR=  os.path.join(RAW_DATA_DIR,"GLES")  

code_df= pd.read_csv(os.path.join(GLES_DIR,r"first_most_imp_coding_list.csv"), sep=',')
coding_list_dict = dict(zip(code_df.Code, code_df.Category))

stata_files = [f for f in os.listdir(GLES_DIR) if f.endswith('.dta') and f.startswith('ZA6838_w') ]
d={}
for file in stata_files:
    try:
        wave = re.search(r'_w(\d+)', file).group(1)
        keyname="kp{}_840_c1".format(wave)
        wave_date = pd.to_datetime(pd.read_stata(os.path.join(GLES_DIR, file), convert_categoricals=False).field_start.iloc[0]).date()
        d[keyname] = {'start_date':wave_date,'fname':file}
    except:
        print('error while processing',file) 
        
        
df_coding_840s= pd.read_csv(os.path.join(RAW_DATA_DIR, r"ZA7957_6838_v2.0.0.csv"), sep=';',encoding='iso-8859-1')

col_value_counts_df=pd.DataFrame()
for col in df_coding_840s.filter(regex='_840_c1').columns:
    col_value_counts = df_coding_840s[col][df_coding_840s[col].ge(0)].value_counts(1)
    #col_value_counts = col_value_counts[col_value_counts.ge(0.001)]
    col_value_counts_df[col]= col_value_counts
col_value_counts_df.index = col_value_counts_df.index.map(coding_list_dict,na_action='ignore')
col_value_counts_df=col_value_counts_df.fillna(0)

a= col_value_counts_df.T.melt(var_name='Category',value_name='Percentage',ignore_index=False).reset_index().rename(columns={'index':'wave'})
a['date']= a.wave.map(d)
a=a.query('date==date')


ZA6838_wa2_sA_v6-0-0.dta


In [9]:
import plotly.express as px

fig = px.line(a, x='date', y='Percentage',color='Category',title='Percentage of Categories in Open-Ended Questions',labels={'Percentage':'Percentage of Categories','wave':'Wave','Category':'Category'})
fig.write_html('wave_class_perc.html')